In [1]:
import pandas as pd

In [2]:
from nsepy import get_history as gh
import datetime as dt

In [3]:

stk_data = pd.read_csv("Tatacoffee13_21.csv")
stk_data['Date'] = pd.to_datetime(stk_data['Date'])

# Filter data for the specified date range
start_date = dt.datetime(2013, 6, 1)
end_date = dt.datetime(2022, 2, 11)

date_range = stk_data[(stk_data['Date'] >= start_date) &(stk_data['Date'] <= end_date)]


In [4]:
date_range

,Date,Open,High,Low,Close
105,2013-06-03,1485.00,1497.50,1451.00,1467.40
106,2013-06-04,1453.00,1509.00,1453.00,1462.15
107,2013-06-05,1449.00,1470.00,1449.00,1459.50
108,2013-06-06,1465.00,1470.00,1455.35,1463.70
109,2013-06-07,1466.20,1484.85,1446.00,1451.70
...,...,...,...,...,...
2220,2021-12-22,202.90,207.80,201.35,205.00
2221,2021-12-23,206.00,206.85,202.05,202.95
2222,2021-12-24,203.90,203.90,199.35,201.00
2223,2021-12-27,200.00,222.00,196.00,218.35


In [5]:
stk_data=stk_data[["Open","High","Low","Close"]]
stk_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    2225 non-null   float64
 1   High    2225 non-null   float64
 2   Low     2225 non-null   float64
 3   Close   2225 non-null   float64
dtypes: float64(4)
memory usage: 69.7 KB


In [6]:
#column="Close"

In [7]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (2225, 4)


In [8]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [9]:
len(data1)

2225

In [10]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

1780
X_train length: (1780, 4)
X_test length: (445, 4)
y_train length: (1780, 4)
y_test length: (445, 4)


In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [13]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [14]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']


In [15]:

perf, result, pred = cominbation(data1, listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -41.57633250372664
BIC:  -41.52447100279195

Order = 2
AIC:  -41.89965675983836
BIC:  -41.80627099960222

Order = 3
AIC:  -41.896681081226916
BIC:  -41.761739857287346

Order = 4
AIC:  -41.89556182371673
BIC:  -41.71903389353323

Order = 5
AIC:  -41.92461661731288
BIC:  -41.70647070014239

Order = 6
AIC:  -42.006769549318946
BIC:  -41.74697432615165

Order = 7
AIC:  -42.06304859245069
BIC:  -41.76157270594566

Order = 8
AIC:  -42.05891183647389
BIC:  -41.71572389089462

Order = 9
AIC:  -42.07506774212317
BIC:  -41.690136303272986

Order = 10
AIC:  -42.096303169891954
BIC:  -41.66959676504925



In [16]:
data1

,Open,High,Low,Close
0,0.849900,0.845408,0.856100,0.854203
1,0.856394,0.967399,0.861040,0.974481
2,0.988480,0.996439,0.984959,0.986240
3,0.985483,0.968105,0.960760,0.956749
4,0.955669,0.975319,0.955033,0.967132
...,...,...,...,...
2220,0.095842,0.096329,0.096510,0.097323
2221,0.097777,0.095745,0.096951,0.096041
2222,0.096466,0.093934,0.095252,0.094821
2223,0.094031,0.105047,0.093143,0.105673


In [17]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.088105,11,28
